#### Split file for mbatch

In [5]:
import pandas as pd

In [7]:
ab = pd.read_csv(
    '../output/dbl-cncr/pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
        sep = '\t', index_col = 0)

In [8]:
# mbatch takes non-negative numbers
ab.iloc[:, 2:].min().min()

0.0

In [46]:
ab.System.value_counts()

System
CPTAC        251
cell_line    132
Name: count, dtype: int64

In [47]:
ab.Cancer_type.value_counts()

Cancer_type
pancreatic_adeno    195
lung_adeno          188
Name: count, dtype: int64

In [105]:
cancers = 'pancreatic-adeno+lung-adeno'

In [49]:
modality = 'transcriptomics'

In [27]:
src_ref = 'src'
sys = 'cell_line'
cl = ab[ab.System == 'cell_line'].copy()
cl.drop(columns = ['System'], inplace = True)
cl.drop(columns = ['Cancer_type'], inplace = True)
cl = cl.T
cl.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [28]:
cl_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)

In [ ]:
cl_transposed_read_test

In [32]:
src_ref = 'ref'
sys = 'cptac'
cp = ab[ab.System == 'CPTAC'].copy()
cp.drop(columns = ['System'], inplace = True)
cp.drop(columns = ['Cancer_type'], inplace = True)
cp = cp.T
cp.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [ ]:
cp_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)
cp_transposed_read_test

In [ ]:
ab

In [51]:
ab.System.value_counts()

System
CPTAC        251
cell_line    132
Name: count, dtype: int64

#### Attach 2 label sets

In [52]:
ls # this is mbatch_i-o.ipynb

dta_ldr.ipynb           mbatch_i-o.ipynb        schedule_vae_gpu.sh*
frnt_end_clf_ump.ipynb  nb_dvl/                 vae.py*
job_scheduler.sh*       plots/
mbatch/                 schedule_vae_cpu.sh*


In [53]:
ls mbatch/bk_mbatch/data/mbatch/

EBNPlus_TrainAndValidateReplicates_Structures/
correctedResults_src_v1.tsv
input_ref_v1.tsv
input_src_v1.tsv


In [54]:
cd = pd.read_csv(
    'mbatch/bk_mbatch/data/mbatch/correctedResults_src_v1.tsv',
        sep = '\t', index_col = 0)

In [ ]:
cd

In [75]:
src_df = cd.T[cd.T.index.str.endswith(".src")]
ref_df = cd.T[cd.T.index.str.endswith(".ref")]

In [ ]:
src_df

In [ ]:
ref_df

In [78]:
ab_src = ab[ab.System == 'cell_line']
ab_ref = ab[ab.System == 'CPTAC']

In [ ]:
ab_src

In [ ]:
ab_ref

In [84]:
src_df.insert(0, 'Cancer_type', list(ab_src.Cancer_type))
ref_df.insert(0, 'Cancer_type', list(ab_ref.Cancer_type))

In [85]:
src_df.insert(0, 'System', 'cell_line')
ref_df.insert(0, 'System', 'CPTAC')

In [86]:
mb_sys = pd.concat([src_df, ref_df], axis = 0)

In [ ]:
mb_sys

#### Run random forest from front-end on mb_sys

In [93]:
systems = 'cell-line+CPTAC'

In [94]:
cancer, modality, systems

('pancreatic-adeno+lung-adeno', 'transcriptomics', 'cell-line+CPTAC')

In [95]:
mb_sys.to_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t')

In [96]:
obj_n = pd.read_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t', index_col = 0)

In [98]:
del(mb_sys)

In [100]:
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import statistics

In [104]:
obj_n

,System,Cancer_type,entrz_tran_1,entrz_tran_10,entrz_tran_100,entrz_tran_1000,entrz_tran_10000,entrz_tran_10001,entrz_tran_10002,entrz_tran_10003,...,entrz_tran_9987,entrz_tran_9988,entrz_tran_9989,entrz_tran_999,entrz_tran_9990,entrz_tran_9991,entrz_tran_9992,entrz_tran_9993,entrz_tran_9994,entrz_tran_9997
smpl_id_1059.src,cell_line,pancreatic_adeno,0.000000,0.000000,8.428773,0.000000,10.508928,9.545096,4.365030,5.634868,...,13.897711,9.041862,11.737195,15.044714,13.003528,14.045384,0.000000,11.128922,11.281378,8.764360
smpl_id_106.src,cell_line,pancreatic_adeno,3.002471,2.357975,8.456246,8.887311,21.357082,9.805549,1.772865,7.907552,...,13.246723,9.038039,11.997689,18.829927,11.636760,14.074575,2.044405,12.273810,10.879885,8.953531
smpl_id_108.src,cell_line,pancreatic_adeno,13.042896,3.958463,7.572169,2.372596,4.529093,10.736176,5.330946,4.659613,...,12.847602,9.981506,13.721504,20.063970,14.411336,14.248902,2.569320,13.708955,11.436215,10.906998
smpl_id_11.src,cell_line,pancreatic_adeno,10.154071,3.845136,9.037572,8.276126,11.288107,9.875335,4.706958,7.404365,...,13.364835,8.557933,12.445397,19.021951,10.587163,14.250355,2.444056,11.352824,10.189871,11.087634
smpl_id_113.src,cell_line,pancreatic_adeno,9.554470,2.671732,9.536537,20.531764,19.299877,9.010785,2.103781,19.532407,...,12.742044,9.151514,12.186347,17.788367,13.716108,12.125570,6.786660,12.063572,10.383208,9.393239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
smpl_id_2783.ref,CPTAC,lung_adeno,4.910000,3.240000,7.240000,9.960000,12.190000,9.780000,3.240000,7.450000,...,13.220000,9.300000,12.590000,12.370000,11.000000,12.900000,3.810000,11.350000,10.590000,10.120000
smpl_id_2784.ref,CPTAC,lung_adeno,4.550000,1.860000,7.650000,9.240000,11.810000,9.720000,3.290000,8.870000,...,13.390000,7.995000,12.760000,13.040000,11.170000,12.880000,2.180000,11.490000,10.490000,10.520000
smpl_id_2785.ref,CPTAC,lung_adeno,4.930000,1.650000,6.590000,11.030000,11.590000,10.600000,3.550000,8.240000,...,13.680000,9.635000,12.510000,12.990000,12.340000,13.230000,3.000000,11.340000,11.020000,9.875000
smpl_id_2786.ref,CPTAC,lung_adeno,4.670000,2.050000,8.080000,9.500000,11.510000,9.580000,4.760000,7.350000,...,13.640000,8.720000,14.100000,13.780000,11.110000,13.250000,1.940000,11.810000,10.710000,10.020000


In [101]:
X = obj_n.iloc[:, 2:]
y_sys = obj_n.System
y_c_typ = obj_n.Cancer_type

sys_scores = []
c_typ_scores = []

for i in list(range(0,9)):
    X_train, X_test, y_train, y_test = train_test_split(X, y_sys)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    sys_scores.append(score)

    y_train = y_c_typ[y_c_typ.index.isin(y_train.index)]
    y_test = y_c_typ[y_c_typ.index.isin(y_test.index)]
    
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    c_typ_scores.append(score)

In [106]:
cancers+'_'+modality+'_'+systems

'pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC'

In [107]:
file_n = cancers+'_'+modality+'_'+systems

In [ ]:
sys_mean = statistics.mean(sys_scores)
c_typ_mean = statistics.mean(c_typ_scores)

sys_err = statistics.stdev(sys_scores)
c_typ_err = statistics.stdev(c_typ_scores)

fig, ax = plt.subplots()

# Create bars
bar1 = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
bar2 = ax.bar('c typ', c_typ_mean, yerr=c_typ_err, capsize = 7, label='Cancer type')

plt.title(file_n) # automate later
plt.xlabel('Cancer Model System')
plt.ylabel('F1 score')

# Legend
ax.legend()
plt.savefig('../results/mbatch/figures/'+file_n+'rf_barplot.png')